# Tools and Agents

--------------------------------------------------------------------------------------------

### 1. Tools :

* Website to choose Tools : https://python.langchain.com/v0.1/docs/integrations/tools/

* Tools in langchain helps us integrate the Gen AI app with multiple external resources or tools like extenal databases, API's, Web scraping and even custom tools etc....

* You can call these tools during the conversation to get real-time data or perform actions.

### 2. Agents

* Agents are decision-makers that use tools to complete a task.

* They can choose which tool to use, when to use it, and how to interact with it based on the input they receive.

* So, this helps to manage task effectively by making choices based on context

### Example 

##### Imagine you're building a chatbot that answers questions about weather:

* Tool: An API that gives real-time weather data.

* Agent: The agent decides when to call the weather API based on the user's question and then sends the result back.

------------------------------------------------------------------------------------------

# Search Engine With Tools and Agents

##### Pre-Requiste

* Install "arxiv"
* Install "wikipedia"

In [19]:
# To Import the .env file
import os
from dotenv import load_dotenv
load_dotenv()
import openai

os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN") 
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

#### Creating Tool (wikipedia)

In [4]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper # these wrappers are needed to run ArxivQueryRun, WikipediaQueryRun 

In [5]:
# Used the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max = 250) # top k represents the number of output, chars max represnt how much charcter used to respond.
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)

In [15]:
# Used the inbuilt tool of arxiv
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max = 250)
arxiv = ArxivQueryRun(api_wrapper = api_wrapper_arxiv)

##### Create Custom tools [RAG Tool]

In [12]:
from langchain_community.document_loaders import WebBaseLoader 
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
documents = documents.split_documents(docs)

vectordb = FAISS.from_documents(documents, HuggingFaceEmbeddings())

retriever = vectordb.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\jemim\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jemim\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Convert the retriever into tool

In [14]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "langsmith-search","Search any info about langsmith")
retriever_tool.name

'langsmith-search'

##### Combine all 3 tools

In [16]:
tools = [wiki,arxiv,retriever_tool]

##### Run all these tools with Agents and LLM model

In [20]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016E2221A770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016E2221BB20>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [22]:
# another way of prompt template

from langchain import hub
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

##### Agents

In [24]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

##### Execute Agent

In [25]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent = agent, tools = tools, verbose=True)
agent_executor 

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [26]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'Tell me about Langsmith',
 'output': "It seems that LangSmith is a platform for building production-grade Large Language Model (LLM) applications. It provides features such as observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their applications, iterate on prompts, and ship quickly with confidence.\n\nSome of the key features of LangSmith include:\n\n* Observability: LangSmith allows developers to analyze traces, configure metrics, and set up dashboards and alerts to monitor their application's performance.\n* Evaluation: LangSmith provides a set of tools to build and run high-quality evaluations, allowing developers to assess the performance of their application and collect human feedback to improve it.\n* Prompt Engineering: LangSmith provides a set of tools to enable and facilitate prompt engineering, allowing developers to find the perfect prompt for their application.\n\nIt also seems that LangSmith is closely integrated w

In [27]:
agent_executor.invoke({"input":"what is machine learning in 2 lines"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning in 2 lines'}`


Page: List of datasets for machine-learning research
e datasets are used in machine learning (ML) research and have been cited in peer-reviewed academic journals. Datasets are an integral part of the field of machine learning. Major adva
Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Here's a 2-line summary based on the results:ficial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit

Machine learning is a field of artificial intelligence that enables algorithms to learn from data and make predictions or decisions without being explicitly programmed. It involves developing statistical models that can generalize to new, unseen data and perform tasks such as classification, regression, clustering, and more.

> Finished chain.


{'input': 'what is machine learning in 2 lines',
 'output': "Here's a 2-line summary based on the results:\n\nMachine learning is a field of artificial intelligence that enables algorithms to learn from data and make predictions or decisions without being explicitly programmed. It involves developing statistical models that can generalize to new, unseen data and perform tasks such as classification, regression, clustering, and more."}

In [28]:
agent_executor.invoke({"input":"are you using my Chat History to answer my question.. say yes or no"})



> Entering new AgentExecutor chain...
no

> Finished chain.


{'input': 'are you using my Chat History to answer my question.. say yes or no',
 'output': 'no'}

------------------------------------------------------------------------------------------